In [ ]:
##???
%load_ext autoreload
%autoreload 2

In [2]:
#Set up packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import categorical_embedder as ce

from sklearn.preprocessing import StandardScaler

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [22]:
#Load data
df = pd.read_csv('../data/raw/beer_reviews.csv')
#Load saved processed data
#traindf = pd.read_csv('../data/processed/traindf.csv')
#target = pd.read_csv('../data/processed/target.csv')

In [ ]:
#EDA
#df.head()
#df.info()
#df.describe()
#df.shape()

In [ ]:
#Find out how many different labels we are predicting
#df.brewery_name.unique()

In [ ]:
#Visualisations

#boxplot = df.boxplot(column=['review_overall', 'review_aroma', 'review_appearance','review_palate','review_taste'])
#boxplot_abv = df.boxplot(column=['beer_abv'])

#df.hist(column=['review_overall', 'review_aroma', 'review_appearance','review_palate','review_taste'])

In [28]:
#Prepare data  - removed inplace=True as it wasn't saving to csv
traindf = df.copy()
traindf = traindf.dropna()
traindf.reset_index(drop=True)
target = traindf['beer_style']
feature = traindf.drop(['brewery_id','review_time','review_profilename','beer_name','beer_abv','beer_beerid','beer_style'], axis=1)


#Save locally
#feature.to_csv('../data/processed/feature.csv')
#target.to_csv('../data/processed/target.csv')

In [4]:
# ce.get_embedding_info identifies the categorical variables, # of unique values and embedding size and returns a dictionary
embedding_info = ce.get_embedding_info(feature)
embedding_info



{'brewery_name': (5155, 50)}

In [5]:
# ce.get_label_encoded_data integer label encodes the categorical variables and prepares it to feed it to neural network. C
#Can do next step of training shallow neural network for embeddings
traindf_encoded,encoders = ce.get_label_encoded_data(feature)

#traindf_encoded.head()



,Unnamed: 0,brewery_name,review_overall,review_aroma,review_appearance,review_palate,review_taste
0,0,4886,1.5,2.0,2.5,1.5,1.5
1,1,4886,3.0,2.5,3.0,3.0,3.0
2,2,4886,3.0,2.5,3.0,3.0,3.0
3,3,4886,3.0,3.0,3.5,2.5,3.0
4,4,1360,4.0,4.5,4.0,4.0,4.5


In [52]:
#featuredf = pd.DataFrame(traindf_encoded)
#featuredf.head
cols = [1,2,3,4,5,6]
featuredf = traindf_encoded[traindf_encoded.columns[cols]]


In [53]:
featuredf.head

<bound method NDFrame.head of          brewery_name  review_overall  review_aroma  review_appearance  \
0                4886       -3.239994     -2.511302          -2.198210   
1                4886       -1.148720     -1.792233          -1.384289   
2                4886       -1.148720     -1.792233          -1.384289   
3                4886       -1.148720     -1.073164          -0.570368   
4                1360        0.245463      1.084042           0.243553   
...               ...             ...           ...                ...   
1518473          4617        1.639646      0.364974          -0.570368   
1518474          4617        0.245463      1.803111          -2.198210   
1518475          4617        0.942555     -0.354095          -1.384289   
1518476          4617        0.245463      1.084042           1.057473   
1518477          4617        1.639646      1.084042           1.057473   

         review_palate  review_taste  
0            -3.317561     -3.162309  
1  

In [31]:
##ERROR HERE -PROCEEDED WITH LABEL ENCODING FIRST

from sklearn.model_selection import train_test_split

# ce.get_embeddings trains NN, extracts embeddings and return a dictionary containing the embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info, 
                            is_classification=True, epochs=100,batch_size=256)

UnimplementedError:  Cast string to float is not supported
	 [[node mean_squared_error/Cast (defined at /opt/conda/lib/python3.7/site-packages/categorical_embedder/__init__.py:175) ]] [Op:__inference_train_function_1721]

Function call stack:
train_function


In [14]:
#Pre-processing: Scaling. (Can turn this into a pipeline - after standardscaler, some reviews turned negative
##cat_cols = ['brewery_name']
num_cols = ['review_overall', 'review_aroma', 'review_appearance','review_palate','review_taste']

#Instantiate scaler and one hot encoder
##ohe = OneHotEncoder(sparse=False,handle_unknown='ignore')
sc = StandardScaler()

#Fit and transform columns
traindf_encoded[num_cols] = sc.fit_transform(traindf_encoded[num_cols])


In [55]:
#from src.data.splitsets import split_sets_random (ModuleNotFoundError: No module named 'src.data')

X_data, X_test, y_data, y_test = train_test_split (featuredf, target, test_size=0.2, random_state=8)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=8)




In [56]:
#Initial run - Save split datasets
np.save('../data/processed/X_train', X_train)
np.save('../data/processed/X_val',   X_val)
np.save('../data/processed/X_test',  X_test)
np.save('../data/processed/y_train', y_train)
np.save('../data/processed/y_val',   y_val)
np.save('../data/processed/y_test',  y_test)

#Subsequent runs - load saved datasets

#X_train = np.load('../data/processed/X_train.npy')
#X_val   = np.load('../data/processed/X_val.npy'  )
#X_test  = np.load('../data/processed/X_test.npy' )
#y_train = np.load('../data/processed/y_train.npy')
#y_val   = np.load('../data/processed/y_val.npy'  )
#y_test  = np.load('../data/processed/y_test.npy' )



,brewery_name,review_overall,review_aroma,review_appearance,review_palate,review_taste
1190393,2292,-0.451628,-1.073164,-0.570368,-1.109519,-1.103587
943422,458,-0.451628,-0.354095,0.243553,-1.109519,-1.103587
591582,263,0.942555,1.803111,0.243553,0.362510,0.955134
1246018,146,-1.148720,-1.073164,-1.384289,-1.109519,-1.103587
246920,745,-1.148720,-0.354095,0.243553,-1.109519,-1.103587
...,...,...,...,...,...,...
1124841,1793,0.245463,0.364974,0.243553,0.362510,1.641374
403267,2237,1.639646,1.803111,0.243553,1.098524,0.955134
484654,4058,-2.542902,-2.511302,-2.198210,-1.845533,-3.848549
173185,3480,0.942555,-1.073164,-2.198210,0.362510,-0.417347


In [ ]:
from joblib import dump
dump(reg,  '../models/linear_poly_2.joblib')

In [ ]:
#Spare

#df.loc[df['brewery_id'] == 27]